# LangChain: Models, Prompts and Output Parsers


## Outline

 * API calls through LangChain:
   * Simple Calls
   * Prompts
   * Models
   * Output parsers

In [0]:
#!pip install openai
!pip install -Uq langchain tiktoken

In [0]:
import openai
import os

from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
#from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

import tiktoken

## Exploring Azure OpenAI

#### Let's start by comparing model responses with direct API calls to Azure OpenAI.

#### Defining Endpoint, Key and Provider

In [0]:
openai.api_type = "azure"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#openai.api_base = "https://chatgpt-summarization.openai.azure.com/"
openai.api_key = os.environ["OPENAI_API_KEY"]

#### Choosing text-davinci-003 model

In [0]:
openai_model_name = "text-davinci-003"
openai_deploy_name = "model-text-davinci-003"
openai.api_version = "2023-09-15-preview"

llm = AzureOpenAI(deployment_name=openai_deploy_name,
                  model_name=openai_model_name,
                  openai_api_key=openai.api_key,
                  openai_api_version=openai.api_version,
                  #prompt=prompt,
                  #max_tokens=800,
                  temperature=0,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0,
                  best_of=1,
                  verbose=True,
                  )

A simple query to the model:

In [0]:
response = llm("Can you tell me how much is 1+1?")
print(response)

#### Choosing gpt-35-turbo model

In [0]:
openai_model_name = "gpt-35-turbo"
openai_deploy_name = "model-gpt-35-turbo"
openai.api_version = "2023-07-01-preview"

llm = AzureOpenAI(deployment_name=openai_deploy_name,
                  model_name=openai_model_name,
                  openai_api_key=openai.api_key,
                  openai_api_version=openai.api_version,
                  #prompt=prompt,
                  #max_tokens=800,
                  temperature=0,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0,
                  best_of=1,
                  verbose=True,
                  )

The same simple query to the model:

In [0]:
response = llm("Can you tell me how much is 1+1?")
print(response)

## Prompts using formatted strings 

In [0]:
openai_model_name = "text-davinci-003"
openai_deploy_name = "model-text-davinci-003"
openai.api_version = "2023-09-15-preview"

llm = AzureOpenAI(deployment_name=openai_deploy_name,
                  model_name=openai_model_name,
                  openai_api_key=openai.api_key,
                  openai_api_version=openai.api_version,
                  #prompt=prompt,
                  #max_tokens=800,
                  temperature=0,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0,
                  best_of=1,
                  verbose=True,
                  )

In [0]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [0]:
response = llm(prompt)
print(response)

## Chat API : LangChain  

#### Loading the ´gpt-35-turbo´ model and the ChatOpenAI wrapper from Langchain

In [0]:
openai.api_type = "azure"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
openai.api_key = os.environ["OPENAI_API_KEY"]

openai_model_name = "gpt-35-turbo"
openai_deploy_name = "model-gpt-35-turbo"
openai.api_version = "2023-07-01-preview"

In [0]:
chat = AzureChatOpenAI(openai_api_base=openai.api_base,
                       openai_api_version=openai.api_version,
                       deployment_name=openai_deploy_name,
                       openai_api_key=os.environ["OPENAI_API_KEY"],
                       openai_api_type=openai.api_type,
                       )


chat

### Prompt template

In [0]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [0]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [0]:
prompt_template.messages[0].prompt

In [0]:
prompt_template.messages[0].prompt.input_variables

In [0]:
customer_style = """American English \
in a calm and respectful tone
"""

In [0]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [0]:
customer_messages = prompt_template.format_messages(style=customer_style,
                                                    text=customer_email)

In [0]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [0]:
print(customer_messages[0])

In [0]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

In [0]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [0]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [0]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [0]:
service_response = chat(service_messages)
print(service_response.content)

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [0]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [0]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [0]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [0]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

In [0]:
type(response.content)

In [0]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary; 'gift' is a string

try:
  response.content.get('gift')
except AttributeError as error:
    print(error)

### Parse the LLM output string into a Python dictionary

In [0]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [0]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [0]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [0]:
format_instructions = output_parser.get_format_instructions()

In [0]:
print(format_instructions)

In [0]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [0]:
print(messages[0].content)

In [0]:
response = chat(messages)
print(response.content)

In [0]:
output_dict = output_parser.parse(response.content)

In [0]:
output_dict

In [0]:
type(output_dict)

In [0]:
output_dict.get('delivery_days')